In [0]:
%sql

CREATE OR REPLACE TABLE odace.map.silver_zones_attraction AS 

WITH cleaned_bronze AS (
  SELECT
    CODGEO,
    LIBGEO,
    AAV2020,
    regexp_replace(LIBAAV2020, '(?i)\\s*\\(partie française\\)', '') AS LIBAAV2020_CLEAN,
    LIBAAV2020,
    CATEAAV2020,
    DEP,
    REG
  FROM odace.map.bronze_zones_attraction
  WHERE AAV2020 != "000"
),
exploded_bronze AS (
  SELECT
    CODGEO,
    LIBGEO,
    AAV2020,
    trim(value) AS LIBAAV2020_CLEAN,
    CATEAAV2020,
    DEP,
    REG
  FROM cleaned_bronze
  LATERAL VIEW explode(split(LIBAAV2020_CLEAN, '\\s* - \\s*')) t as value
),
name_cleaned AS (
  SELECT
    CODGEO,
    LIBGEO,
    AAV2020,
    CASE 
      WHEN regexp_replace(LIBAAV2020_CLEAN, 'œ', 'oe') ILIKE "Hesdin%" THEN "Hesdin"
      WHEN regexp_replace(LIBAAV2020_CLEAN, 'œ', 'oe') ILIKE "Cugand%" THEN "Cugand"
      ELSE regexp_replace(LIBAAV2020_CLEAN, 'œ', 'oe')
    END AS LIBAAV2020,
    CATEAAV2020,
    DEP,
    REG
  FROM exploded_bronze
)

SELECT 
  b.CODGEO AS CODGEO,
  b.LIBGEO AS LIBGEO,
  b.AAV2020 AS CODEAAV,
  g.CODGEO AS CODGEOAAV,
  b.LIBAAV2020,
  b.CATEAAV2020 AS CATEAAV,
  b.DEP as DEP,
  b.REG as REG
FROM name_cleaned b
JOIN odace.map.silver_geo g
  ON regexp_replace(upper(b.LIBAAV2020), '[^A-Z0-9]', '') = regexp_replace(upper(regexp_replace(g.LIBGEO, 'œ', 'oe')), '[^A-Z0-9]', '')
WHERE b.CODGEO NOT LIKE CONCAT('%', g.CODGEO, '%')